In [19]:
%run 'Common.ipynb'

In [20]:
from wndcharm.FeatureSpace import FeatureSpace
f=FeatureSpace.NewFromFileOfFiles( os.path.expanduser('data/medial_7_6_8_delta_3bin.fof'), long=True )
f.ToFitFile('data/medial_7_6_8_delta_3bin.fit')

IOError: [Errno 2] No such file or directory: 'data/medial_7_6_8_delta_3bin.fof'

In [9]:
dmat = f.ContiguousDataMatrix()
print (dmat)
print (f.data_list)
print (f._contiguous_ground_truth_labels)
print (f._contiguous_ground_truth_values)

[[  2.47100000e+03   9.96700000e-03   9.09090000e-02 ...,  -5.68879000e+04
    1.24726000e+05   7.11482000e-01]
 [  2.46700000e+03   8.28920000e-02   4.14750000e-02 ...,  -5.34658000e+04
    1.29016000e+05   7.17171000e-01]
 [  2.30200000e+03   8.79350000e-02   1.46444000e-01 ...,  -6.22379000e+04
    1.28003000e+05   7.28210000e-01]
 ..., 
 [  2.40200000e+03   7.63360000e-02   1.62866000e-01 ...,  -6.03631000e+04
    1.30804000e+05   7.21693000e-01]
 [  2.45000000e+03   1.28302000e-01   7.62710000e-02 ...,  -5.37427000e+04
    1.35765000e+05   7.17335000e-01]
 [  2.43000000e+03   1.49250000e-02   4.86660000e-02 ...,  -4.15241000e+04
    1.06458000e+05   7.17794000e-01]]
[array([[  2.47100000e+03,   9.96700000e-03,   9.09090000e-02, ...,
         -5.68879000e+04,   1.24726000e+05,   7.11482000e-01],
       [  2.46700000e+03,   8.28920000e-02,   4.14750000e-02, ...,
         -5.34658000e+04,   1.29016000e+05,   7.17171000e-01],
       [  2.30200000e+03,   8.79350000e-02,   1.46444000e-0

In [10]:
print (f.class_names)
class_vals = [float(x) for x in f.class_names]
print (class_vals)

['1', '6', '4', '7', '2', '5', '3', '8']
[1.0, 6.0, 4.0, 7.0, 2.0, 5.0, 3.0, 8.0]


In [9]:
dat_mat = np.mgrid[0:8,0:5]
print (dat_mat)
(train,test) = round_robin_iteration (19,dat_mat)
print ('-------')
print (test)
print (train)

[[[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]
  [5 5 5 5 5]
  [6 6 6 6 6]
  [7 7 7 7 7]]

 [[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]]
-------
[array([[2, 2, 2, 2, 2]]), array([[0, 1, 2, 3, 4]])]
[array([[3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5],
       [6, 6, 6, 6, 6],
       [7, 7, 7, 7, 7],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1]]), array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])]


In [ ]:
nfeatures = 200
niter = np.product ( [m.shape[0] for m in f.data_list] )
class_vals = [float(x) for x in f.class_names]
n_classes = len(class_vals)
n_correct = np.asarray( [0]*n_classes)
(train,test) = round_robin_iteration (0,f.data_list)
print ('Features          : {}'.format(nfeatures))
print ('Train class sizes : {}'.format([x.shape[0] for x in train]))
print ('Classes           : {}'.format(n_classes))
print ('Iterations        : {}'.format(niter))
for iter_idx in range ( niter ):
    # Split
    (train,test) = round_robin_iteration (iter_idx,f.data_list)
    (contig_train_mat, contig_train_vals) = list_to_contig_mat (train, class_vals)
    (contig_test_mat, contig_test_vals) = list_to_contig_mat (test, class_vals)

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
    preds,pred_vals = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)

#    preds = rand_forest_clf (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
#    preds = rand_forest_clf (sorted_train[:,:nfeatures], np.asarray([norm_test[1][:,:nfeatures],sorted_test[0][:,:nfeatures]]), contig_train_vals, iter_idx)
    for pred_idx in range (len(preds)):
        if (preds[pred_idx] == contig_test_vals[pred_idx]):
            n_correct[pred_idx] += 1
    cumul_acc = [(float(x) / (float(iter_idx)+1.0)) for x in n_correct]
    print ('Iteration {}; Predictions: {}; cumulative accuracies: {}'.format(iter_idx, preds, cumul_acc))


    200 features, RFC
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.73125, 0.7294642857142857]
    200 features, WND5
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.7366071428571429, 0.7598214285714285]
    100 features, WND5
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.75, 0.7205357142857143]


In [14]:
dmat = f.ContiguousDataMatrix()
print (dmat)
print (f.data_list)
class_vals = [float(x) for x in f.class_names]
contig_mat, mat_vals = list_to_contig_mat (f.data_list, class_vals)
print (contig_mat)

print (f._contiguous_ground_truth_labels)
print (mat_vals)

[[  2.47100000e+03   9.96700000e-03   9.09090000e-02 ...,  -5.68879000e+04
    1.24726000e+05   7.11482000e-01]
 [  2.46700000e+03   8.28920000e-02   4.14750000e-02 ...,  -5.34658000e+04
    1.29016000e+05   7.17171000e-01]
 [  2.30200000e+03   8.79350000e-02   1.46444000e-01 ...,  -6.22379000e+04
    1.28003000e+05   7.28210000e-01]
 ..., 
 [  2.40200000e+03   7.63360000e-02   1.62866000e-01 ...,  -6.03631000e+04
    1.30804000e+05   7.21693000e-01]
 [  2.45000000e+03   1.28302000e-01   7.62710000e-02 ...,  -5.37427000e+04
    1.35765000e+05   7.17335000e-01]
 [  2.43000000e+03   1.49250000e-02   4.86660000e-02 ...,  -4.15241000e+04
    1.06458000e+05   7.17794000e-01]]
[array([[  2.47100000e+03,   9.96700000e-03,   9.09090000e-02, ...,
         -5.68879000e+04,   1.24726000e+05,   7.11482000e-01],
       [  2.46700000e+03,   8.28920000e-02,   4.14750000e-02, ...,
         -5.34658000e+04,   1.29016000e+05,   7.17171000e-01],
       [  2.30200000e+03,   8.79350000e-02,   1.46444000e-0

In [18]:
dmat = f.ContiguousDataMatrix()
class_vals = [float(x) for x in f.class_names]
contig_mat, mat_vals = list_to_contig_mat (f.data_list, class_vals)

nfeatures = 100
niter = contig_mat.shape[0]
n_classes = len(class_vals)
print ('Train class sizes : {}'.format(contig_mat.shape[0]-1))
print ('Features          : {}'.format(nfeatures))
print ('Iterations        : {}'.format(niter))
predictions = []
actual = []
for iter_idx in range ( niter ):
    # Split
    contig_train_mat = np.delete(contig_mat,[iter_idx],axis=0)
    contig_test_mat = np.asarray([contig_mat[iter_idx]])

    contig_train_vals = np.delete (mat_vals, [iter_idx])
    contig_test_vals = np.asarray ([mat_vals[iter_idx]])

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
    preds,pred_val = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)
    #pred_val = rand_forest_reg (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
    #pred_val = lin_reg (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)
    predictions.append (pred_val[0])
    actual.append (mat_vals[iter_idx])
    print ('Iteration {}; Predictions: {}; actual: {}'.format(iter_idx, predictions[-1], actual[-1]))

score, p_value = pearsonr(predictions, actual)
score *= score
print ('R^2: {}, p-value: {}'.format (score, p_value))


Train class sizes : 39
Features          : 100
Iterations        : 40
Iteration 0; Predictions: 2.89350219858; actual: 1.0
Iteration 1; Predictions: 3.66774272383; actual: 1.0
Iteration 2; Predictions: 3.37823950968; actual: 1.0
Iteration 3; Predictions: 2.81087974926; actual: 1.0
Iteration 4; Predictions: 4.56147283239; actual: 1.0
Iteration 5; Predictions: 4.26043003709; actual: 1.0
Iteration 6; Predictions: 2.77135725128; actual: 6.0
Iteration 7; Predictions: 2.92580796111; actual: 6.0
Iteration 8; Predictions: 4.29713198227; actual: 6.0
Iteration 9; Predictions: 3.3896427624; actual: 6.0
Iteration 10; Predictions: 3.84069959243; actual: 4.0
Iteration 11; Predictions: 4.68481399976; actual: 4.0
Iteration 12; Predictions: 4.32191891858; actual: 4.0
Iteration 13; Predictions: 3.90841158299; actual: 4.0
Iteration 14; Predictions: 2.58138946648; actual: 4.0
Iteration 15; Predictions: 2.7808477818; actual: 7.0
Iteration 16; Predictions: 2.89843866244; actual: 7.0
Iteration 17; Prediction

    200 features RandForReg
    R^2: 0.100336411037, p-value: 0.0464318824205
    20 features lin reg:
    R^2: 0.0491493694884, p-value: 0.169178264743
    20 features RandForReg
    R^2: 0.0595653398403, p-value: 0.129095037849
    100 features RandForReg
    R^2: 0.0395069277481, p-value: 0.218869033287
    100 features RandForReg - binned, 8 bins.
    R^2: 0.0197249101125, p-value: 0.387377038132


